# Part 1: Reconstruction of a positive wavefunction

## Getting Started

The following imports are needed to run this tutorial:

In [6]:
import torch
import numpy as np

from qucumber.binary_rbm import BinaryRBM
from qucumber.quantum_reconstruction import QuantumReconstruction
from qucumber.positive_wavefunction import PositiveWavefunction

from qucumber.callbacks import MetricEvaluator

import qucumber.utils.data as data

import quantum_ising_chain as TFIM # for calculating observables after training. More later...

'''
#from rbm_tutorial import RBM_Module, BinomialRBM
import torch
#from observables_tutorial import TFIMChainEnergy, TFIMChainMagnetization
import numpy as np
import csv
%matplotlib inline
import sys
sys.path.append('../../qucumber/')
from positive_wavefunction import PositiveWavefunction
from quantum_reconstruction import QuantumReconstruction
sys.path.append('../../qucumber/utils/')
import utils.training_statistics as ts
sys.path.append('../../qucumber/utils/ed/')
from hamiltonians import *
from data_generator import *
import quantum_ising_chain as TFIM
#import importlib.util
#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook
'''

ModuleNotFoundError: No module named 'qucumber'

The _BinaryRBM_ class in *binary_rbm.py* contains the generic properties of an RBM with a binary visible and hidden layer (e.g. it's effective energy and sampling the hidden and visible layers). 

The actual quantum wavefunction reconstruction occurs in the _QuantumReconstruction_ class in *quantum_reconstruction.py*. A _QuantumReconstruction_ object is initialized with a neural network state (in this case, a PositiveWavefunction object).

*callbacks.py* contains functions that allow the user to evaluate the quality of the training (i.e. based on the fidelity or KL divergence).

## Training

Let's beging with training the RBM on a positive wavefunction. We consider the quantum Ising model with Hamiltonian $H=-J\sum_{\langle i j \rangle} S^z_i S^z_j - h \sum_i S^x_i$
at its quantum critical point $h/J=1$.  The training data has been generated and is contained in the file *tfim1d_N10_train_samples.txt*.  It contains 10,000 measurements of the $S^z$ states of 10 qubits, represented as zeros or ones.

To evaluate how well the RBM is training, we compute the fidelity, $|\langle \psi|\psi_{\rm RBM} \rangle|^2$, between the true wavefunction of the system and the wavefunction the RBM reconstructs. First, we need to load our training data and the true wavefunction of this system.

In [7]:
train_samples_path = 'tfim1d_train_samples.txt'
psi_path           = 'tfim1d_psi.txt'

train_samples,target_psi = data.load_data(tr_sam_path,psi_path)

The following arguments are required to construct a **PositiveWavefunction** neural network state:

1. **The number of visible units**. This is 10 for the case of our dataset.
2. **The number of hidden units in the hidden layer of the RBM**. This number is set to the number of visible units by default (10 in the case of our dataset).

In [3]:
nv = train_samples.shape[-1]
nh = nv

nn_state = PositiveWavefunction(num_visible=nv,num_hidden=nh)

Now we can specify the parameters of the training process:

1. **epochs**: the number of epochs, i.e. training cycles that will be performed; 1000 should be fine
2. **batch_size**: the number of data points used in the positive phase of the gradient; we'll go with 100
3. **num_chains**: the number of data points used in the negative phase of the gradient. Keeping this larger than the *batch_size* is preferred; we'll go with 200
4. **CD**: the number of contrastive divergence steps; CD=1 seems to be good enough in most cases
5. **lr**: the learning rate; we will use a learning rate of 0.01 here
6. **log_every**: how often you would like the program to update you during the training; we choose 50 - that is, every 50 epochs the program will print out the fidelity. This parameter is required in the *MetricEvaluator*.

In [1]:
epochs     = 1000
batch_size = 100
num_chains = 200
CD         = 1
lr         = 0.01
log_every  = 50

Once we initialize the parameters of the *PositiveWavefunction* and the *MetricEvaluator*, we can now begin training. Our *QuantumReconstruction* object, *qr* (see below), contains a function called *fit* that executes the training process.

In [17]:
nn_state.space = nn_state.generate_Hilbert_space(nv) # generate the entire visible space of the system.
callbacks      = [MetricEvaluator(log_every,{'Fidelity':ts.fidelity,'KL':ts.KL},target_psi=target_psi)]

qr = QuantumReconstruction(nn_state)

qr.fit(train_samples, epochs, batch_size, num_chains, CD,lr, progbar=False,callbacks=callbacks)

NameError: name 'qr' is not defined

## After Training 

After training the RBM, the *fit* function will have stored the trained weights and biases. Now, we can generate samples from the trained RBM and calculate physical observables.

We have created an example python file called *quantum_ising_chain.py* that takes advantage of QuCumber's built in *observables* module to calculate the energy of a TFIM. A tutorial for utilizing the *observables* module for different systems (i.e. not a TFIM) will be supplied in the near future.

*quantum_ising_chain* (imported as *TFIM*) comprises of a class called *TransverseFieldIsingChain*. A *TransverseFieldIsingChain* is instantiated with the following arguments:

1. **hx**: This is the *h* value of the TFIM (1 in the case of our dataset).
2. **n_measurements**: The number of samples to be generated from the trained RBM. These samples will be used to calculate the observables.

In [20]:
n_measurements = 50000
hx             = 1
tfim           = TFIM.TransverseFieldIsingChain(hx,n_measurements)

NameError: name 'TFIM' is not defined

To go ahead and calculate the energy and magnetization of the 50,000 generated samples, the *tfim* object has a function called *run* which will output a dictionary containing the energy (key: 'energy') and the magnetization (key: 'sigmaZ'). 

In [ ]:
simulation = tfim.Run(nn_state,n_eq=200, show_convergence=True)

step = list(range(n_eq))

Energy     = simulation['energy']
err_energy = np.std(Energy)/np.sqrt(Energy.shape[0])

sigmaZ = simulation['sigmaZ']
err_sZ = np.std(sigmaZ)/np.sqrt(sigmaZ.shape[0])

ax1 = plt.axes()
ax1.plot(step, Energy, color='red')
ax1.fill_between(step, Energy-err_energy, Energy+err_energy, color='red', alpha=0.4) 
#ax1.plot(step, true_energy, color='black')
ax1.grid()
ax1.set_xlim(0,n_eq)
ax1.set_xlabel('CD Step')
ax1.set_ylabel('Energy')

ax2 = plt.axes()
ax2.plot(step, sigmaZ, color='red')
ax2.fill_between(step, sigmaZ-err_sZ, sigmaZ+err_sZ, color='red', alpha=0.4) 
#ax2.plot(step_list, true_mag, color='black')
ax2.grid()
ax2.set_xlim(0,n_eq)
ax2.set_xlabel('CD Step')
ax2.set_ylabel('Magnetization')

You can see a brief transient period in each observable, before the state of the machine "warms up" to equilibrium.  After that, the values fluctuate around the mean.  The exact value for the energy is -1.2785, and for the magnetization is 0.7072.